# RAG on HTML documents


## Step-1: Configuration

In [1]:
from my_config import MY_CONFIG

## Step-2: Setup Embeddings

In [2]:
# If connection to https://huggingface.co/ failed, uncomment the following path
import os
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

In [3]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings

Settings.embed_model = HuggingFaceEmbedding(
    model_name = MY_CONFIG.EMBEDDING_MODEL
)

## Step-3: Connect to Milvus

In [4]:
# connect to vector db
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.vector_stores.milvus import MilvusVectorStore

vector_store = MilvusVectorStore(
    uri = MY_CONFIG.DB_URI ,
    dim = MY_CONFIG.EMBEDDING_LENGTH , 
    collection_name = MY_CONFIG.COLLECTION_NAME,
    overwrite=False  # so we load the index from db
)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

print ("✅ Connected to Milvus instance: ", MY_CONFIG.DB_URI )

2025-01-16 21:48:32,682 [DEBUG][_create_connection]: Created new connection using: 057b8aeb18704c05b5888f7bfe39d58c (async_milvus_client.py:600)


✅ Connected to Milvus instance:  ./rag_html.db


## Step-4: Load Document Index from DB

In [5]:
%%time

from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store, storage_context=storage_context)

print ("✅ Loaded index from vector db:", MY_CONFIG.DB_URI )

✅ Loaded index from vector db: ./rag_html.db
CPU times: user 98.1 ms, sys: 12.5 ms, total: 111 ms
Wall time: 118 ms


## Step-5: Setup LLM

In [6]:
from llama_index.llms.replicate import Replicate
from llama_index.core import Settings

llm = Replicate(
    model= MY_CONFIG.LLM_MODEL,
    temperature=0.1
)

Settings.llm = llm

## Step-6: Query

In [7]:
query_engine = index.as_query_engine()
res = query_engine.query("What is AI Alliance?")
# res = query_engine.query("What is Apache license?")
print(res)

The AI Alliance is a global community of leading organizations from industry, academia, research, and government, collaborating to advance open, safe, and responsible AI. It aims to foster an open community and enable developers and researchers to accelerate responsible innovation in AI while ensuring scientific rigor, trust, safety, security, diversity, and economic competitiveness. The AI Alliance plans to start or enhance projects that meet various objectives, such as developing benchmarks and evaluation standards, advancing open foundation models, fostering a vibrant AI hardware accelerator ecosystem, supporting global AI skills building, and launching initiatives that encourage open development of AI in safe and beneficial ways.


In [8]:
query_engine = index.as_query_engine()
res = query_engine.query("Where was the demo night held?")
# res = query_engine.query("What are some Apache software?")
print(res)

The demo night was held at Shack15 in San Francisco, CA.


In [9]:
query_engine = index.as_query_engine()
res = query_engine.query("When was the moon landing?")
print(res)

The context information does not provide information on the moon landing. Therefore, I cannot provide an answer to that query.
